In [ ]:
# Section 1: Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import Audio
from scipy.io import wavfile
import soundfile as sf
import tensorflow as tf
import tensorflow_hub as hub
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import librosa

# Section 2: Define Data Path
data_path = "data\\noiseData"

# Section 3: Define Audio Sampling Function
from scipy.io import wavfile
import scipy.signal
def ensure_sample_rate(original_sample_rate, waveform, desired_sample_rate=22000):
    if original_sample_rate != desired_sample_rate:
        desired_length = int(
            round(float(len(waveform))/original_sample_rate * desired_sample_rate))
        waveform = scipy.signal.resample(waveform, desired_length)
    return desired_sample_rate, waveform

# Section 4: Load YAMNet Model and Define Pipeline
# Load YAMNet model
model_yamnet = hub.load('https://tfhub.dev/google/yamnet/1')

# Define target classes
classes = ["Bicycle", "MotorCycle", "Private car", "Pickup", "SUV", "PSV", "Buses", 
           "Light trucks", "Medium trucks", "Heavy trucks", "Others"]

# Function to read and preprocess audio
def read_audio(file_path, desired_sample_rate=22000):
    audio, sample_rate = librosa.load(file_path, sr=None)  # Load with original sample rate
    sample_rate, audio = ensure_sample_rate(sample_rate, audio, desired_sample_rate)
    return audio, sample_rate

# Improved pipeline function
def pipeline(file_path):
    audio_data, sample_rate = read_audio(file_path)
    # Reshape audio to 2D if needed (mono channel)
    if len(audio_data.shape) > 1:
        audio_data = np.mean(audio_data, axis=1)
    audio_data = tf.convert_to_tensor(audio_data, dtype=tf.float32)
    scores, embeddings, spectrogram = model_yamnet(audio_data)
    # Use mean embedding if variable length, no fixed padding
    mean_embedding = tf.reduce_mean(embeddings, axis=0)
    # Placeholder mapping based on YAMNet scores
    class_scores = tf.reduce_mean(scores, axis=0).numpy()
    top_class_index = tf.argmax(class_scores).numpy() % len(classes)  # Modulo to fit 11 classes
    return classes[top_class_index]

# Section 5: Update Metadata with Predictions
import pandas as pd
import os

# Load the metadata CSV
metadata_path = "data\\DATA\\output_chunks\\metadata.csv"
metadata_df = pd.read_csv(metadata_path)

# Function to classify audio and update metadata
def update_classes(metadata_df):
    for index, row in metadata_df.iterrows():
        audio_file = row['slice_file_name']
        full_path = os.path.join("data\\\\DATA\\\\output_chunks", audio_file)  # Preserve path
        if os.path.exists(full_path):
            try:
                predicted_class = pipeline(full_path)
                # Update the class column with prediction
                metadata_df.at[index, 'class'] = predicted_class
            except Exception as e:
                print(f"Error processing {audio_file}: {e}")
                metadata_df.at[index, 'class'] = "Others"  # Fallback
        else:
            print(f"File not found: {full_path}")
            metadata_df.at[index, 'class'] = "Others"  # Fallback
    return metadata_df

# Apply the classification and update
updated_metadata_df = update_classes(metadata_df)

# Save the updated metadata back to CSV
updated_metadata_df.to_csv("updated_metadata.csv", index=False)
print("Updated metadata saved to 'updated_metadata.csv'")


c:\Users\USER\Desktop\SonusAI\venv\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-0.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-1.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-2.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-3.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-4.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-5.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-6.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-7.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-8.wav
File not found: data\\DATA\\output_chunks\Around Arya School(1°16_32_ S 36°49_29_ E)--1-9.wav
File not found: data\\DATA\\output_chunks\Around Arya School

: 